In [74]:
from scipy import signal 
import numpy as np
import timeit

import arlpy.uwapm as pm
import arlpy.plot as aplt

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from bokeh.models.annotations import Title
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.palettes import Category10,viridis,Spectral11
from bokeh.models import ColumnDataSource, Range1d, BoxAnnotation, HoverTool, LogColorMapper, LogTicker, ColorBar
output_notebook()

samplefreq = 100*10**3 #Hz
time = 0.0
def impulse_response(h,r,d1,d2,samplefreq, fc, **kwargs):
    Type = kwargs.get('Type', 'valid')
    
    ssp = [
    [ 0, 1540],  # 1540 m/s at the surface
    [h/4, 1530],  # 1530 m/s at 10 m depth
    [h/2, 1532],  # 1532 m/s at 20 m depth
    [3*h/4, 1533],  # 1533 m/s at 25 m depth
    [h, 1535]   # 1535 m/s at the seabed
    ]
    
    surface = np.array([[r, 0.5+0.5*np.sin(2*np.pi*0.05*r)] for r in np.linspace(0,1000,1001)])
    maxAngle = 180*(np.arctan(h/(r/2)))/np.pi 
    env = pm.create_env2d(depth = h, rx_range = r, tx_depth = d1, rx_depth = d2, frequency = fc, max_angle = maxAngle, min_angle = -maxAngle, surface = surface, soundspeed = ssp)
    arrivals = pm.compute_arrivals(env)
    Hn = pm.arrivals_to_impulse_response(arrivals,samplefreq,abs_time = True)
    
    if Type == 'valid':
        return Hn.real
    if Type == 'full':
        return env, arrivals, Hn
    
def tapmigratedIR(h, r, d1, d2, samplefreq, fc):
    Hn_avg = []
    maxLength = 0
    for i in range (40):
        H = np.random.normal(h,2)
        R = np.random.normal(r,2)
        D1 = np.random.normal(d1)
        D2 = np.random.normal(d2)
        Hn = impulse_response(H,R,D1,D2,samplefreq,fc)
        if len(Hn) > maxLength:
            maxLength = len(Hn)
        Hn_avg.append(Hn.real)


    for i in range (len(Hn_avg)):
        Hn_avg[i] = np.append(Hn_avg[i],(np.zeros(maxLength-len(Hn_avg[i]))))

    Hn_final = [np.mean(value) for value in zip(*Hn_avg)]
    count = 0
    for j in range(len(Hn_final)):
        if Hn_final[j] == 0:
            count += 1
        else:
            break

    Hn_final = Hn_final[count:]
    
    return Hn_final

def multiFrequencyResponse(H,R,D1,D2_list,samplefreq, fc, bw, **kwargs):
    global time
    """
    This function is able to compare different frequency responses of a given channel characterstis
    from varying the vertical position of the receiver, in this case D2.
    
    """
    Type = kwargs.get('Type', 'multi')
    
    lowfreq = fc-(bw/2.0)
    highfreq = fc+(bw/2.0)

    freq_final_list = []
    max_final_new_list = []
    
    if Type == 'multi':
        for D2 in D2_list:
            Hn = impulse_response(H,R,D1,D2,samplefreq,fc)
            max_new_list = np.abs(np.fft.rfft(Hn))
            signalSize = len(Hn)
            freq_l = np.fft.fftfreq(signalSize,1000/samplefreq)
            count_low, count_high = 0,0
            #max_new_list = [max_new_list[idx] for idx, val in enumerate(freq_l) if val >= lowfreq and val <= highfreq]
            #freq_list = [i for i in freq_l if i >= lowfreq and i <= highfreq]
            start, end = -1, -1
            for i in range(len(freq_l)):
                if freq_l[i] >= lowfreq:
                    start = i
                    break
            for i in range(start, len(freq_l)):
                if freq_l[i] >= highfreq or freq_l[i] < 0:
                    end = i
                    break
                #if freq_l[i] <= lowfreq and freq_l[i] >=0:
                #    count_low += 1
                #if freq_l[i] >= highfreq or freq_l[i] <=0:
                #    count_high += 1

            max_new_list = max_new_list[start:end]
            freq_list = freq_l[start:end]
            #max_new_list = max_new_list[count_low:len(max_new_list)-count_high]
            #freq_list = freq_l[count_low:len(freq_l)-count_high]
            
            #max_new_list = max_new_list[int(freq_list[0]):len(freq_list)+int(freq_list[0])]#-int(freq_list[0])
            #freq_list=[freq for freq in freq_l if freq>=lowfreq and freq<=highfreq]

            #freq_list = freq_list[:(len(max_new_list))]
            freq_final_list.append(freq_list)
            max_final_new_list.append(max_new_list)
            
        return freq_final_list,max_final_new_list,freq_list
    
    if Type == 'duo':
        Hn1 = tapmigratedIR(H,R,D1,D2_list[0],samplefreq,fc)
        signalSize = len(Hn1)
        freq_list = np.fft.fftfreq(signalSize,1000/samplefreq)
        #freq_list1=[freq for freq in freq_l if freq>=lowfreq and freq<=highfreq]
        max_new_list1 = np.abs(np.fft.rfft(Hn1))
        
        count_low, count_high = 0,0
        for i in range(len(freq_list)):
            if freq_list[i]<=lowfreq:
                count_low += 1
            if freq_list[i] >= highfreq:
                count_high += 1

        max_new_list1 = max_new_list[count_low:count_high]
        freq_list1 = freq_l[count_low:count_high]
        
        #print(freq_list1)
        #print((freq_list1[0]), freq_list1[len(freq_list1)])
        #max_new_list1 = max_new_list[int(freq_list1[0]):len(freq_list1)+int(freq_list1[0])]#-int(freq_list[0])]
        mean1 = np.mean(max_new_list1)
        A = [(max_value-mean1) for max_value in max_new_list1]
        
        freq_list1 = freq_list1[:(len(A))]
        freq_final_list.append(freq_list1)
        max_final_new_list.append(A)



        Hn2 = tapmigratedIR(H,R,D1,D2_list[1],samplefreq,fc)
        signalSize = len(Hn2)
        freq_l = np.fft.fftfreq(signalSize,1000/samplefreq)
        freq_list2=[freq for freq in freq_l if freq>=lowfreq and freq<=highfreq]
        max_new_list = np.abs(np.fft.rfft(Hn2))
        max_new_list2 = max_new_list[int(freq_list2[0]):len(freq_list2)+int(freq_list2[0])]#-int(freq_list[0])]
        mean2 = np.mean(max_new_list2)
        B = [(max_value-mean2) for max_value in max_new_list2]
        
        freq_list2 = freq_list2[:(len(max_new_list2))]
        freq_final_list.append(freq_list2)
        max_final_new_list.append(B)
        
        CMNV = sum(a*b for a, b in zip(A, B))/(np.sqrt((sum(a**2 for a in A)*sum(b**2 for b in B))))
        CMNS = sum(a**2 for a in A)/(np.sqrt((sum(a**2 for a in A)*sum(a**2 for a in A))))
        return freq_final_list,max_final_new_list,CMNV, CMNS


Loading BokehJS ...

In [75]:
H, R,D1,fc,bw = 100,100,5,20,30
D2_list = np.linspace(6,6.2,2)
tic = timeit.default_timer()
freq_final_list, max_final_new_list, freq_list = multiFrequencyResponse(H,R,D1,D2_list,samplefreq, fc, bw)
toc = timeit.default_timer()

roundD2_test = ['%.2f'% elem for elem in D2_list]
data = {'xs': freq_final_list,
        'ys': max_final_new_list,
        'labels': roundD2_test,
        'mypalette': viridis(len(D2_list))}

source = ColumnDataSource(data)

FI = figure(x_axis_label = "Frequency(kHz)", y_axis_label = "Ampltidue",plot_width=950, plot_height=400,logo = None)
FI.title.text = 'Frequency Response of the Channel of range {0} m'.format(R)
FI.title.align = "center"
FI.title.text_color = "black"
FI.title.text_font_size = "15px"

FI.multi_line(xs='xs', ys='ys', legend='labels', line_color = 'mypalette',source=source)
print("time taken (s): ", toc-tic)
show(FI)

time taken (s):  0.2807464669967885
